# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.interpret import ExplanationClient
from azureml.pipeline.steps import AutoMLStep


In [2]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


### Workspace 

In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="automl-udacity-capstone")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-134856
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-134856


### Compute Cluster

In [4]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

ws = Workspace.from_config() # this automatically looks for a directory .azureml

#Create compute cluster
cluster_name="automl-project"

try:
  compute_target=ComputeTarget(workspace=ws,name=cluster_name)
  print("Found existing compute target")
except ComputeTargetException:
  print('Creating a new compute target')
  compute_config=AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                       max_nodes=5, 
                                                       idle_seconds_before_scaledown=2400,
                                                       vm_priority='lowpriority')
  compute_target=ComputeTarget.create(ws,cluster_name,compute_config)
  

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes =  2)

Creating a new compute target
Creating
Succeeded....................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [5]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

Capstone-Udacity ComputeInstance Succeeded
hd-project AmlCompute Succeeded
automl-project AmlCompute Succeeded


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [8]:
# Import dataset

from azureml.core import Dataset

ds = Dataset.get_by_name(ws, name='Glass Dataset')
final_df = ds.to_pandas_dataframe()
final_df

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


In [20]:
from azureml.core import Dataset
from azureml.data.dataset_factory import DataType

# create a TabularDataset from a delimited file behind a public web url and convert column "Survived" to boolean
web_path ="https://raw.githubusercontent.com/Aishwaryasasanapuri/test2/main/glass.csv"
final_df = Dataset.Tabular.from_delimited_files(path=web_path) #, set_column_types={'Type': DataType.to_bool()})

# preview the first 3 rows of titanic_ds
final_df.take(3).to_pandas_dataframe()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [21]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_hours" : 0.3,
    "enable_early_stopping" : True,
    #"iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 8,
    #"max_cores_per_iteration": -1,
    "n_cross_validations": 5,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    #"verbosity": logging.INFO,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             training_data = final_df,
                             label_column_name = 'Type',
                             iterations= 100,
                             **automl_settings
                            )

In [22]:
# TODO: Submit your experiment
automl_run = exp.submit(automl_config)

Running on remote.


In [23]:
automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-udacity-capstone,AutoML_ee7fc2f8-27f1-4d94-89bf-199778c1e0e2,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [24]:
from azureml.widgets import RunDetails
RunDetails(run).show()

# Wait for the remote run to complete
automl_run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'AutoML_ee7fc2f8-27f1-4d94-89bf-199778c1e0e2',
 'target': 'automl-project',
 'status': 'Completed',
 'startTimeUtc': '2021-01-16T11:21:08.999712Z',
 'endTimeUtc': '2021-01-16T11:43:09.101842Z',
 'properties': {'num_iterations': '100',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'automl-project',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-udacity-capstone","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group":"aml-quickstarts-134856","workspace_name":"quick-starts-ws-134856","region":"southcentralus","compute_target":"automl-project","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":100,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":n

In [25]:
automl_run.get_metrics('accuracy')

{'accuracy': 0.798671096345515}

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.


Retrieve the Best Model's explanation from the best_run which includes explanations for engineered features and raw features. Make sure that the run for generating explanations for the best model is completed.

In [28]:
# Wait for the best model explanation run to complete
from azureml.core.run import Run
model_explainability_run_id = automl_run.id + "_" + "ModelExplain"
print(model_explainability_run_id)
model_explainability_run = Run(experiment=exp, run_id=model_explainability_run_id)
model_explainability_run.wait_for_completion()

# Get the best run object
automl_best_run, model = automl_run.get_output()
automl_best_run_metrics = automl_best_run.get_metrics()

AutoML_ee7fc2f8-27f1-4d94-89bf-199778c1e0e2_ModelExplain


In [29]:
print(model)
print('Best Run Id: ', automl_best_run.id)
print('\n Accuracy:', automl_best_run_metrics['accuracy'])

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_impurity_split=None,
                                                                                                    min_samples_leaf=0.01,
                                                                                                    min_samples_split=0.01,
                    

#### Describing the Best model

In [30]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['4', '6', '3', '22', '20'],
 'weights': [0.42857142857142855,
             0.14285714285714285,
             0.14285714285714285,
             0.14285714285714285,
             0.14285714285714285]}

4 - minmaxscaler
{'copy': True, 'feature_range': (0, 1)}

4 - randomforestclassifier
{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'log2',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 0.01,
 'min_samples_split': 0.01,
 'min_weight_fraction_leaf': 0.0,
 'n_estim

In [31]:
#TODO: Save the best model
import joblib
#best_model_automl=
joblib.dump(model,filename='outputs/best_automlmodel.pkl')

['outputs/best_automlmodel.pkl']

In [32]:
automl_best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":5}',
 'ensembled_iterations': '[4, 6, 3, 22, 20]',
 'ensembled_algorithms': "['RandomForest', 'RandomForest', 'ExtremeRandomTrees', 'ExtremeRandomTrees', 'RandomForest']",
 'ensemble_weights': '[0.42857142857142855, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285]',
 'best_individual_pipeline_score': '0.9370217590919065',
 'best_individual_iteration': '4',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_ee7fc2f8-27f1-4d94-89bf-199778c1e0e2_ModelExplain',
 'model_explanation': 'True'}

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [33]:
from azureml.core.model import Model

model = Model.register(workspace = ws,
                        model_path ='outputs/best_automlmodel.pkl',
                        model_name = "automl")

Registering model automl


In [34]:
model.id

'automl:1'

TODO: In the cell below, send a request to the web service you deployed to test it.

In [52]:
model_name = automl_best_run.properties['model_name']

#env = automl_best_run.get_environment().save_to_directory(path='environments')

script_file_name = 'score.py'

automl_best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'Score.py')

In [53]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

myenv=Environment(name="env")
conda_packages = ['numpy']
pip_packages = ['azureml-sdk','azureml-defaults']
mycondaenv = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages, python_version='3.6.2')
myenv.python.conda_dependencies=mycondaenv
myenv.register(workspace=ws)

inference_config = InferenceConfig(entry_script='Score.py',source_directory='.',environment=myenv)

#inference_config = InferenceConfig(entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1,
                                               enable_app_insights=True, 
                                               auth_enabled=True, 
                                               description = 'Service for Automl Classification')

#aci_service_name = 'automl-capstone'

aci_service_name = 'aumlcapstone'

print(aci_service_name)

aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

aumlcapstone
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: b5c892d5-c656-43a8-9e29-a99275967100
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: aumlcapstone. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image f2210b748ee647f897fc87d35888691d.azurecr.io/azureml/azureml_8809d06a4e01ac1626dd4002f2e4a1b7 locally. Please refer to https://aka.ms/debugimage#service-

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: b5c892d5-c656-43a8-9e29-a99275967100
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: aumlcapstone. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image f2210b748ee647f897fc87d35888691d.azurecr.io/azureml/azureml_8809d06a4e01ac1626dd4002f2e4a1b7 locally. Please refer to https://aka.ms/debugima

In [54]:
aci_service.get_logs()

In [57]:
print(aci_service.get_logs())

None


In [49]:
print("scoring URI: " + aci_service.scoring_uri)
print("Swagger URI: " + aci_service.swagger_uri)
print("Authetication Key: " + aci_service.get_keys()[0])

TypeError: must be str, not NoneType

#### Call the services

In [50]:
import requests
import json

# URL for the web service
scoring_uri = aci_service.swagger_uri #'<your web service URI>'
# If the service is authenticated, set the key or token
key =  aci_service.get_keys()[0]  #'<your key or token>'

# Two sets of data to score, so we get two results back
data = {"data":
        [
            {
               'RI': 5,
               'Na': 2,
               'Mg': 3.5,
               'Al': 0.7,
               'Si': 83,
               'K': 1.2,
               'Ca': 9,
               'Ba': 0.12,
               'Fe': 0.15
               
               },
            {
               'RI': 2,
               'Na': 25,
               'Mg': 2.5,
               'Al': 1.7,
               'Si': 63,
               'K': 2.2,
               'Ca': 6,
               'Ba': 0.15,
               'Fe': 0.18   
           }
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

MissingSchema: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?

### Testing the deployed service

In [51]:
%run endpoint.py

NameError: name 'aci_service' is not defined

In [ ]:
data_test = final_df[210:215]

In [ ]:
y_true = data_test.pop('Type')
sample_json = json.dumps({'data':data_test.to_dict(orient='records')})
print(sample_json)

In [ ]:
output = aci_service.run(sample_json)
print('Prediction: ', output)
print('True Values: ', y_true.values)

### Delete a Web Service

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
aci_service.delete()

#### Deleting compute

In [ ]:
compute_target.delete()